# Lab 1

### Daniel Kim

In [1]:
# importing arcpy files
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\ArcToolbox\\Scripts')
import arcpy
arcpy.env.overwriteOutput = True

# set up geojson environment variables 
from subprocess import call
import os
os.environ["GDAL_DATA"] = "C:\\OSGeo4W\\share\\gdal"
os.environ["GDAL_DRIVER_PATH"] = "C:\\OSGeo4W\\bin\\gdalplugins"
os.environ["PROJ_LIB"] = "C:\\OSGeo4W\\share\\proj"
os.environ["PATH"] = "C:\\OSGeo4W\\bin;"+os.environ["PATH"]+";C:\\OSGeo4W\\apps\\msys\\bin;C:\\OSGeo4W\\apps\\Python27\\Scripts"


# setting up workspace
arcpy.env.workspace = "H:/GEOG458"

# Setting up feature class 
census = "saep_bg10/saep_bg10.shp"
census1 = "saep_bg10copy.shp"
table = "WashingtonFIPS.dbf"

In [2]:
# Made a copy of the census feature class and copied features into copied shapefile
arcpy.management.CopyFeatures(census, census1)

<Result 'H:/GEOG458\\saep_bg10copy.shp'>

#### Problem 1: For each county in Washington, create a GeoJSON file whose filename is the county name and whose contents are the polygons for the block groups within that county. 

#### This means that you will end up with as many GeoJSON files as there are counties in WA. Each such file should be named after a county and contain only the block groups from that county.

Add a field to the census data that will represent the county name corresponding to each row

In [3]:
# add field called CountyName in census block feature class
arcpy.management.AddField(census1, "CountyName", "STRING")

# create cursor to update new field
cursor = arcpy.da.UpdateCursor(census1, ["COUNTYFP10", "CountyName"])

# creating cursor to search through table to patch county id to census block feature class
cursor1 = arcpy.da.SearchCursor(table, ["FIPSCounty", "CountyName"])

# updates and adds county name from table to feature class
for row in cursor:
    for row1 in cursor1:
        if (row[0] == row1[0]):
            row[1] = row1[1]
    cursor1.reset()
    cursor.updateRow(row)
    
# delete cursors
del cursor
del cursor1

Create a list of county names

In [4]:
# create a search cursor from table
cursor = arcpy.da.SearchCursor(table, ["CountyName"])

# define a list
name_list = []

# insert coutny names into list
for row in cursor:
    name_list.append(row[0])
    
# delete cursor
del cursor

Convert census block feature class into separate feature classes representing each county in WA

In [5]:
# performs feature class to feature class conversion on all the different county names based on census block feature class
for name in name_list:
    arcpy.FeatureClassToFeatureClass_conversion(census1, 
                                                "H:/GEOG458/featureclasses/",
                                               name,
                                               '"CountyName" = ' + "'%s'" %name)

Convert shapefiles to geojson files

In [6]:
# loops through, searching for each shapefile of the county names and convert them to geojson format
for name in name_list:
    call(['C:\\OSGeo4W\\bin\\ogr2ogr', 
          '-f','GeoJSON','-t_srs','WGS84', 
          '-s_srs','EPSG:2927', 
          'H:/GEOG458/geojson_results/' + name + '.geojson', 
          'H:/GEOG458/featureclasses/' + name + '.shp'])

#### Print out a ranked list, in descending order, of the ten largest total populations in 2017 of counties in Washington, according to this dataset.

In [7]:
# Create search cursor to add 2017 population per county
cursor = arcpy.da.SearchCursor(census1, ["CountyName", "POP2017"])

# Initialize a dictionary
popsum_list = []

# Fill dicionary with sum of population with each county 
for name in name_list:
    sum = 0
    for row in cursor:
        if (row[0] == name):
            sum +=row[1]
    popsum_list.append(sum)
    cursor.reset()

# print to check if popsum_list is correct
print popsum_list

[19870.0, 22290.0, 193499.99999999997, 76829.99999999999, 74240.00000000001, 470999.9999999994, 4100.0, 105900.00000000001, 41420.0, 7740.0, 90330.00000000003, 2200.0, 95630.0, 72970.0, 82790.0, 31360.0, 2153700.0, 264300.00000000006, 44730.0, 21660.0, 77440.00000000003, 10700.0, 63190.00000000001, 42110.00000000001, 21249.999999999996, 13370.0, 859400.0000000003, 16510.0, 124100.00000000001, 11690.0, 789400.0000000005, 499800.0, 44510.0, 276899.9999999999, 4030.0, 61399.999999999985, 216300.00000000003, 48639.99999999999, 253000.0]


In [8]:
# initialize and associate key and pairs of population and county names
County_pop17 = dict(zip(popsum_list, name_list))

# initialize a dictionary to hold top ten greatest populations and county names
top10 = {}

# extract top 10 counties with most population and place in new dictionary
for key in sorted(County_pop17, reverse=True)[:10]:
    top10[key] = County_pop17[key]

# prints from lowest to highest of the top 10 most populated counties in 2017
for key in sorted(top10):
    print "Population: " + str(key) + " County: " + top10[key]

Population: 193500.0 County: Benton
Population: 216300.0 County: Whatcom
Population: 253000.0 County: Yakima
Population: 264300.0 County: Kitsap
Population: 276900.0 County: Thurston
Population: 471000.0 County: Clark
Population: 499800.0 County: Spokane
Population: 789400.0 County: Snohomish
Population: 859400.0 County: Pierce
Population: 2153700.0 County: King


The approach I took initially was to follow along with the last class assignment with setting up my environment and writing variables to the feature classes. I mostly imagnied how I would go about solving the problems on Arcmap. Although that did help with starting, I soon realized that it wasn't that simple. This was a challenging first lab but I feel that I have learned a lot and I am appreciative of the challenge provided through the assignment. Also without the help of Professor Bergmann, I believe that I would not have been able to finish the assignment this quickly.

Overall this assignment took three days to complete and a total of 18 hours to complete. I could have finished it much sooner but due to my lack of experience with coding in python, I experienced a lot of hardship with troubleshooting.